In [1]:
%cd ~/Workspace/kaggle-2019Q3-siim/

/home/chengjiun/Workspace/kaggle-2019Q3-siim


In [6]:
Path.mkdir??

In [ ]:
Path(checkpoint_path).mkdir(exis)

In [2]:
import torch
import torchvision
import fastai
from fastai.vision import Learner, DatasetType, flip_lr, progress_bar
from TOOLS.mask_functions import mask2rle
from utils import seed_everything
from models.fastai_unet_learner import (
    acc_create_opt,
    unet_learner,
    dice,
    set_BN_momentum,
    AccumulateStep,
    dice_overall,
)
from dataset.fastai_data import new_transform, get_data
import numpy as np
import PIL
import pandas as pd
import gc
from functools import partial

sz = 256
bs = 1
n_acc = 64 // bs  # gradinet accumulation steps
nfolds = 1
SEED = 2019

# eliminate all predictions with a few (noise_th) pixesls
noise_th = 75.0 * (sz / 128.0) ** 2  # threshold for the number of predicted pixels
best_thr0 = 0.2  # preliminary value of the threshold for metric calculation

IMAGE_STATS_DICT = {
    128: ([0.615, 0.615, 0.615], [0.291, 0.291, 0.291]),
    256: ([0.540, 0.540, 0.540], [0.264, 0.264, 0.264]),
    512: ([0.529, 0.529, 0.529], [0.259, 0.259, 0.259]),
}


TRAIN = f"input/train_{sz}"
TEST = f"input/test_{sz}"
MASKS = f"input/masks_{sz}"
backbone = torchvision.models.resnet34

from pathlib import Path
checkpoint_path = "checkpoints/fastai-resnet34/"
Path(checkpoint_path).mkdir(exist_ok=True)
stats = IMAGE_STATS_DICT[sz]


# Prediction with flip TTA
def pred_with_flip(
    learn: fastai.basic_train.Learner,
    ds_type: fastai.basic_data.DatasetType = DatasetType.Valid,
):
    # get prediction
    preds, ys = learn.get_preds(ds_type)
    preds = preds[:, 1, ...]
    # add fiip to dataset and get prediction
    learn.data.dl(ds_type).dl.dataset.tfms.append(flip_lr())
    preds_lr, ys = learn.get_preds(ds_type)
    del learn.data.dl(ds_type).dl.dataset.tfms[-1]
    preds_lr = preds_lr[:, 1, ...]
    ys = ys.squeeze()
    preds = 0.5 * (preds + torch.flip(preds_lr, [-1]))
    del preds_lr
    gc.collect()
    torch.cuda.empty_cache()
    return preds, ys

TypeError: mkdir() got an unexpected keyword argument 'exit_ok'

In [ ]:

seed_everything(SEED)


Learner.create_opt = acc_create_opt

fastai.data_block.ItemLists.transform = new_transform

get_data_p = partial(get_data, TRAIN, TEST, stats, sz, bs)
dice_p = partial(dice, best_thr0, noise_th)

In [ ]:

# Display some images with masks
get_data_p(0).show_batch()

In [3]:

unet_learner(get_data_p(0), backbone, metrics=[dice_p]).model

NameError: name 'get_data_p' is not defined

In [4]:
circle_1 = 1
circle_2 = 1
lr_0 = 1e-3
scores, best_thrs = [], []

for fold in range(nfolds):
    print("fold: ", fold)
    data = get_data_p(fold)
    learn = unet_learner(data, backbone, metrics=[dice_p])
    learn.clip_grad(1.0)
    set_BN_momentum(learn.model)

    # fit the decoder part of the model keeping the encode frozen
    lr = lr_0
    learn.fit_one_cycle(circle_1, lr, callbacks=[AccumulateStep(learn, n_acc)])

    # fit entire model with saving on the best epoch
    learn.unfreeze()
    learn.fit_one_cycle(
        circle_2, slice(lr / 80, lr / 2), callbacks=[AccumulateStep(learn, n_acc)]
    )
    learn.save(checkpoint_path+"/fold" + str(fold))

    # prediction on val and test sets
    preds, ys = pred_with_flip(learn)
    pt, _ = pred_with_flip(learn, DatasetType.Test)

    if fold == 0:
        preds_test = pt
    else:
        preds_test += pt

    # convert predictions to byte type and save
    preds_save = (preds * 255.0).byte()
    torch.save(preds_save, checkpoint_path + "/preds_fold" + str(fold) + ".pt")
    np.save(checkpoint_path + "/items_fold" + str(fold), data.valid_ds.items)

    # remove noise
    preds[preds.view(preds.shape[0], -1).sum(-1) < noise_th, ...] = 0.0

    # optimal threshold
    # The best way would be collecting all oof predictions followed by a single threshold
    # calculation. However, it requres too much RAM for high image resolution
    dices = []
    thrs = np.arange(0.01, 1, 0.01)
    for th in progress_bar(thrs):
        preds_m = (preds > th).long()
        dices.append(dice_overall(preds_m, ys).mean())
    dices = np.array(dices)
    scores.append(dices.max())
    best_thrs.append(thrs[dices.argmax()])

    if fold != nfolds - 1:
        del preds, ys, preds_save
    gc.collect()
    torch.cuda.empty_cache()

preds_test /= nfolds

print("scores: ", scores)
print("mean score: ", np.array(scores).mean())
print("thresholds: ", best_thrs)
best_thr = np.array(best_thrs).mean()
print("best threshold: ", best_thr)

fold:  0


NameError: name 'get_data_p' is not defined